In [ ]:
import xarray as xr
from fair.energy_balance_model import multi_ebm, step_temperature
from tqdm.auto import tqdm
import numpy as np
import pooch
import pandas as pd
import matplotlib.pyplot as pl

In [ ]:
# this won't work for anybody except me :)
# if you want this file, run the code at https://github.com/ClimateIndicator/forcing-timeseries/blob/main/notebooks/make-forcing.ipynb

In [ ]:
ds = xr.load_dataset('/nfs/b0110/Users/mencsm/git/ClimateIndicator/forcing-timeseries/output/ERF_ensemble.nc')

In [ ]:
variables = list(ds.keys())

In [ ]:
total     = np.zeros((273, 100000))
natural   = np.zeros((273, 100000))
aerosol   = np.zeros((273, 100000))
wmghg     = np.zeros((273, 100000))
other_ant = np.zeros((273, 100000))

for variable in tqdm(variables):
    #print(variable)
    total = total + ds[variable]
    if variable in ['solar', 'volcanic']:
        natural = natural + ds[variable]
    elif variable not in [
        'aerosol-radiation_interactions', 'aerosol-cloud_interactions',
        'O3', 'H2O_stratospheric', 'contrails', 'BC_on_snow', 'land_use'
    ]:
        wmghg = wmghg + ds[variable]
    elif variable in ['aerosol-radiation_interactions', 'aerosol-cloud_interactions']:
        aerosol = aerosol + ds[variable]
    else:
        other_ant = other_ant + ds[variable]

anthro = total - natural

In [ ]:
np.max(np.abs(total - natural - aerosol - wmghg - other_ant))

In [ ]:
cal_1_2_0_obj = pooch.retrieve(
    "https://zenodo.org/record/8399112/files/calibrated_constrained_parameters.csv",
    known_hash = "md5:de3b83432b9d071efdd1427ad31e9076"
)

In [ ]:
cal = pd.read_csv(cal_1_2_0_obj, index_col=0)

In [ ]:
cal

In [ ]:
cal.loc[:,'clim_c1':'clim_c3']

In [ ]:
ebms = multi_ebm(
    configs = cal.index,
    ocean_heat_capacity = cal.loc[:,'clim_c1':'clim_c3'].values,
    ocean_heat_transfer = cal.loc[:,'clim_kappa1':'clim_kappa3'].values,
    deep_ocean_efficacy = cal['clim_epsilon'].values,
    stochastic_run = [None]*1001,
    sigma_eta = [0.5]*1001,
    sigma_xi = [0.5]*1001,
    gamma_autocorrelation = cal['clim_gamma'].values,
    seed = [14]*1001,
    use_seed = [False]*1001,
    forcing_4co2 = cal['clim_F_4xCO2'].values,
    timestep = 1,
    timebounds = np.arange(1750, 2023),
)

In [ ]:
eb_matrix_d_array = ebms["eb_matrix_d"].data
forcing_vector_d_array = ebms["forcing_vector_d"].data
stochastic_d_array = ebms["stochastic_d"].data

In [ ]:
cummins_state_array = np.zeros((273, 1001, 4))

In [ ]:
for i_timepoint in tqdm(range(273)):
    cummins_state_array[
        i_timepoint + 1 : i_timepoint + 2, ...
    ] = step_temperature(
        cummins_state_array[i_timepoint : i_timepoint + 1, ...],
        eb_matrix_d_array[None, ...],
        forcing_vector_d_array[None, ...],
        stochastic_d_array[i_timepoint : i_timepoint + 1, ...],
        total.data[i_timepoint : i_timepoint + 1, :1001, None],
    )

In [ ]:
cummins_state_array.shape

In [ ]:
pl.plot(cummins_state_array[:, :, 1]);